### This model includes a Binary Quantum classification implementation using QISKIT ON AN 8-Qubit Circuit based on Transfer learning applied on another QISKIT Circuit invoking the QML MOdel Parameters for producing a Quantum State and then the calculation of its State Fidelity with another such implementation to produce a Similarity Score 

In [1]:
import numpy as np
import qiskit
from IPython.display import clear_output
from qiskit import QuantumCircuit
from qiskit.algorithms.optimizers import COBYLA
from qiskit.circuit import ParameterVector, ClassicalRegister
from qiskit.circuit.library import ZFeatureMap
from qiskit.quantum_info import SparsePauliOp
from qiskit.utils import algorithm_globals
from qiskit_machine_learning.algorithms.classifiers import NeuralNetworkClassifier
from qiskit_machine_learning.neural_networks import EstimatorQNN
from sklearn.model_selection import train_test_split
from qiskit.quantum_info import Statevector
from qiskit import Aer, transpile, assemble
from qiskit.visualization import plot_histogram
from qiskit.quantum_info import state_fidelity
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import numpy as np
from PIL import Image
from qiskit import execute

# Assuming 'circuit' is your quantum circuit

algorithm_globals.random_seed = 12345

/tmp/ipykernel_30166/3413930127.py:7: DeprecationWarning: ``qiskit.algorithms`` has been migrated to an independent package: https://github.com/qiskit-community/qiskit-algorithms. The ``qiskit.algorithms`` import path is deprecated as of qiskit-terra 0.25.0 and will be removed no earlier than 3 months after the release date. Please run ``pip install qiskit_algorithms`` and use ``import qiskit_algorithms`` instead.
  from qiskit.algorithms.optimizers import COBYLA
2023-12-16 13:00:57.316380: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-16 13:00:57.383428: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off err

In [2]:
from qiskit import IBMQ

# Replace 'YOUR_API_TOKEN' with your actual API token
IBMQ.save_account('API_Token')
IBMQ.load_account()


/tmp/ipykernel_30166/2883629526.py:4: DeprecationWarning: The package qiskit.providers.ibmq is being deprecated. Please see https://ibm.biz/provider_migration_guide to get instructions on how to migrate to qiskit-ibm-provider (https://github.com/Qiskit/qiskit-ibm-provider) and qiskit-ibm-runtime (https://github.com/Qiskit/qiskit-ibm-runtime).
  IBMQ.save_account('83fb7e1e877c4e10f5bc3867bc38227f8984becd6d53c4ebbb86de3095ba946f89227b0913b8a7e6b6e1cada1fdc89edff7c14641b2d75ea9ce8060e9e4b14e3')
/tmp/ipykernel_30166/2883629526.py:4: DeprecationWarning: The qiskit.IBMQ entrypoint and the qiskit-ibmq-provider package (accessible from 'qiskit.providers.ibmq`) are deprecated and will be removed in a future release. Instead you should use the qiskit-ibm-provider package which is accessible from 'qiskit_ibm_provider'. You can install it with 'pip install qiskit_ibm_provider'. Just replace 'qiskit.IBMQ' with 'qiskit_ibm_provider.IBMProvider'
  IBMQ.save_account('83fb7e1e877c4e10f5bc3867bc38227f89

<AccountProvider for IBMQ(hub='ibm-q', group='open', project='main')>

In [3]:
provider = IBMQ.get_provider(hub='ibm-q')
backend = provider.get_backend('simulator_statevector')


In [4]:
# We now define a two qubit unitary as defined in [3]
def conv_circuit(params):
    target = QuantumCircuit(2)
    target.rz(-np.pi / 2, 1)
    target.cx(1, 0)
    target.rz(params[0], 0)
    target.ry(params[1], 1)
    target.cx(0, 1)
    target.ry(params[2], 1)
    target.cx(1, 0)
    target.rz(np.pi / 2, 0)
    return target


# Let's draw this circuit and see what it looks like
params = ParameterVector("θ", length=3)
circuit = conv_circuit(params)
# circuit.draw("mpl")



In [5]:
def conv_layer(num_qubits, param_prefix):
    qc = QuantumCircuit(num_qubits, name="Convolutional Layer")
    qubits = list(range(num_qubits))
    param_index = 0
    params = ParameterVector(param_prefix, length=num_qubits * 3)
    for q1, q2 in zip(qubits[0::2], qubits[1::2]):
        qc = qc.compose(conv_circuit(params[param_index : (param_index + 3)]), [q1, q2])
        qc.barrier()
        param_index += 3
    for q1, q2 in zip(qubits[1::2], qubits[2::2] + [0]):
        qc = qc.compose(conv_circuit(params[param_index : (param_index + 3)]), [q1, q2])
        qc.barrier()
        param_index += 3

    qc_inst = qc.to_instruction()

    qc = QuantumCircuit(num_qubits)
    qc.append(qc_inst, qubits)
    return qc


circuit = conv_layer(4, "θ")
# circuit.decompose().draw("mpl")


In [6]:
def pool_circuit(params):
    target = QuantumCircuit(2)
    target.rz(-np.pi / 2, 1)
    target.cx(1, 0)
    target.rz(params[0], 0)
    target.ry(params[1], 1)
    target.cx(0, 1)
    target.ry(params[2], 1)

    return target


params = ParameterVector("θ", length=3)
circuit = pool_circuit(params)
# circuit.draw("mpl")

In [7]:
def pool_layer(sources, sinks, param_prefix):
    num_qubits = len(sources) + len(sinks)
    qc = QuantumCircuit(num_qubits, name="Pooling Layer")
    param_index = 0
    params = ParameterVector(param_prefix, length=num_qubits // 2 * 3)
    for source, sink in zip(sources, sinks):
        qc = qc.compose(pool_circuit(params[param_index : (param_index + 3)]), [source, sink])
        qc.barrier()
        param_index += 3

    qc_inst = qc.to_instruction()

    qc = QuantumCircuit(num_qubits)
    qc.append(qc_inst, range(num_qubits))
    return qc


sources = [0, 1]
sinks = [2, 3]
circuit = pool_layer(sources, sinks, "θ")
# circuit.decompose().draw("mpl")

In [8]:
feature_map = ZFeatureMap(8)

ansatz = QuantumCircuit(8, name="Ansatz")

# First Convolutional Layer
ansatz.compose(conv_layer(8, "c1"), list(range(8)), inplace=True)

# First Pooling Layer
ansatz.compose(pool_layer([0, 1, 2, 3], [4, 5, 6, 7], "p1"), list(range(8)), inplace=True)

# Second Convolutional Layer
ansatz.compose(conv_layer(4, "c2"), list(range(4, 8)), inplace=True)

# Second Pooling Layer
ansatz.compose(pool_layer([0, 1], [2, 3], "p2"), list(range(4, 8)), inplace=True)

# Third Convolutional Layer
ansatz.compose(conv_layer(2, "c3"), list(range(6, 8)), inplace=True)

# Third Pooling Layer
ansatz.compose(pool_layer([0], [1], "p3"), list(range(6, 8)), inplace=True)

# Combining the feature map and ansatz
circuit = QuantumCircuit(8)
circuit.compose(feature_map, range(8), inplace=True)
circuit.compose(ansatz, range(8), inplace=True)

observable = SparsePauliOp.from_list([("Z" + "I" * 7, 1)])

# we decompose the circuit for the QNN to avoid additional data copying
qnn = EstimatorQNN(
    circuit=circuit.decompose(),
    observables=observable,
    input_params=feature_map.parameters,
    weight_params=ansatz.parameters,
)

In [9]:
classifier = NeuralNetworkClassifier(
    qnn,
    optimizer=COBYLA(maxiter=200),  # Set max iterations here
)

In [10]:
def conv_layer_state(num_qubits, paramvector):
    qc = QuantumCircuit(num_qubits, name="Convolutional Layer")
    qubits = list(range(num_qubits))
    param_index = 0
    params = paramvector
    for q1, q2 in zip(qubits[0::2], qubits[1::2]):
        qc = qc.compose(conv_circuit(params[param_index : (param_index + 3)]), [q1, q2])
        qc.barrier()
        param_index += 3
    for q1, q2 in zip(qubits[1::2], qubits[2::2] + [0]):
        qc = qc.compose(conv_circuit(params[param_index : (param_index + 3)]), [q1, q2])
        qc.barrier()
        param_index += 3

    qc_inst = qc.to_instruction()

    qc = QuantumCircuit(num_qubits)
    qc.append(qc_inst, qubits)
    return qc

In [11]:
def pool_layer_state(sources, sinks, paramvector):
    num_qubits = len(sources) + len(sinks)
    qc = QuantumCircuit(num_qubits, name="Pooling Layer")
    param_index = 0
    params = paramvector
    for source, sink in zip(sources, sinks):
        qc = qc.compose(pool_circuit(params[param_index : (param_index + 3)]), [source, sink])
        qc.barrier()
        param_index += 3

    qc_inst = qc.to_instruction()

    qc = QuantumCircuit(num_qubits)
    qc.append(qc_inst, range(num_qubits))
    return qc


In [12]:
def data_image(label, class_dir):
    x = []
    y = []
    for image_file in range(6):
        image_path = os.path.join(class_dir, f"{image_file+1}.jpeg")
        # image_path = class_dir+'/'+ str(image_file+1)
        # Load and preprocess the image
        image = Image.open(image_path)
        image = image.convert('L')
        # plt.imshow(image)
        image = image.resize((28, 28))  # Resize to the model's input size
        image = np.array(image) / 255.0  # Normalize pixel values

        x.append(image)
        y.append(label)

    x = np.array(x)
    y = np.array(y)
    return x, y

In [13]:
def calculate_fidelity(rho, sigma):
    # Calculate the square root of rho
    sqrt_rho = np.sqrt(rho)
    
    # Calculate the matrix product: sqrt_rho * sigma
    term1 = np.matmul(sqrt_rho, np.matmul(sigma, sqrt_rho))
    
    # Calculate the square root of the result: sqrt(sqrt_rho * sigma * sqrt_rho)
    # print(term1)
    sqrt_result = np.sqrt(term1)
    
    # Calculate the trace of the square root: Tr(sqrt(sqrt_rho * sigma * sqrt_rho))
    trace_sqrt_result = np.trace(sqrt_result)
    
    # Calculate the fidelity: Tr(sqrt(sqrt_rho * sigma * sqrt_rho))^2
    fidelity = np.abs(trace_sqrt_result)**2
    
    return fidelity

In [ ]:
n = 10
with open('fidelity_results_8BIT_RESIZE_counts_ibmqc.txt', 'w') as file:
    for p in range(n):
        x = str(1+p)
        dataset_dir1 = '/home/stavya/Desktop/ML/Cosface/data/S'+x
        custom_x1, custom_y1 = data_image(1+p, dataset_dir1)
        custom_x1.resize(6,8)
        for q in range(n):
            y = str(1+q)
            if x!=y:
                dataset_dir2 = '/home/stavya/Desktop/ML/Cosface/data/S'+y
                custom_x2, custom_y2 = data_image(1+q, dataset_dir2)
                custom_x2.resize(6,8)
                custom_x = np.vstack((custom_x1, custom_x2))
                custom_y = np.append(custom_y1, custom_y2)
                print(custom_x.shape, custom_y.shape)
                
                classifier.fit(custom_x, custom_y)
                def extract_param(prefix, paramvector):
                    if prefix == "c1":
                        for i in range(24):
                            paramvector[i] = classifier.weights[i]
                    
                    if prefix == "p1":
                        for i in range(12):
                            paramvector[i] = classifier.weights[i+24]
                    
                    if prefix == "c2":
                        for i in range(12):
                            paramvector[i] = classifier.weights[i+36]
                    
                    if prefix == "p2":
                        for i in range(6):
                            paramvector[i] = classifier.weights[i+48]

                    if prefix == "c3":
                        for i in range(6):
                            paramvector[i] = classifier.weights[i+54]
                    
                    if prefix == "p3":
                        for i in range(3):
                            paramvector[i] = classifier.weights[i+60]

                    return (paramvector)
                
                paramvec1_c1 = np.zeros(24)
                c1_params1 = extract_param("c1", paramvec1_c1)
                paramvec1_p1 = np.zeros(12)
                p1_params1 = extract_param("p1", paramvec1_p1)
                paramvec1_c2 = np.zeros(12)
                c2_params1 = extract_param("c2", paramvec1_c2)
                paramvec1_p2 = np.zeros(6)
                p2_params1 = extract_param("p2", paramvec1_p2)
                paramvec1_c3 = np.zeros(6)
                c3_params1 = extract_param("c3", paramvec1_c3)
                paramvec1_p3 = np.zeros(3)
                p3_params1 = extract_param("p3", paramvec1_p3)

                c1_circuit = conv_layer_state(8, c1_params1)
                p1_circuit = pool_layer_state([0, 1, 2, 3], [4, 5, 6, 7], p1_params1)
                c2_circuit = conv_layer_state(4, c2_params1)
                p2_circuit = pool_layer_state([0, 1], [2, 3], p2_params1)
                c3_circuit = conv_layer_state(2, c3_params1)
                p3_circuit = pool_layer_state([0], [1], p3_params1)


                # final = np.zeros((59,60))
                for i in range(6):
                    data1 = custom_x1[i]
                    normalized_data1 = (np.array(data1) - np.min(data1)) / (np.max(data1) - np.min(data1))
                    angles1 = 2 * np.pi * normalized_data1
                    feature_map_state1 = ZFeatureMap(8)
                    feature_map_circuit1 = feature_map_state1.bind_parameters(dict(zip(feature_map_state1.parameters, angles1)))
                    ansatz_state1 = QuantumCircuit(8, name="Ansatz")
                    ansatz_state1.compose(c1_circuit, list(range(8)), inplace=True)
                    ansatz_state1.compose(p1_circuit, list(range(8)), inplace=True)
                    ansatz_state1.compose(c2_circuit, list(range(4,8)), inplace=True)
                    ansatz_state1.compose(p2_circuit, list(range(4, 8)), inplace=True)
                    ansatz_state1.compose(c3_circuit, list(range(6, 8)), inplace=True)
                    ansatz_state1.compose(p3_circuit, list(range(6, 8)), inplace=True)
                    full_circuit1 = QuantumCircuit(8)
                    full_circuit1.compose(feature_map_circuit1, range(8), inplace=True)
                    full_circuit1.compose(ansatz_state1, range(8), inplace=True)
                    c1 = ClassicalRegister(1, name='c')
                    full_circuit1.add_register(c1)
                    full_circuit1.measure(7, 0)
                    transpiled_circuit = transpile(full_circuit1, backend)

                    # Execute the transpiled circuit on the target backend
                    job = execute(transpiled_circuit, backend, shots=1024)

                    # Get the result
                    result = job.result()
                    counts = result.get_counts()


                    num_qubits = 1
                    density_matrices_individual_qubits = {qubit: np.zeros((2, 2), dtype=complex) for qubit in range(num_qubits)}

                    # Calculate density matrices for individual qubits
                    total_shots = sum(counts.values())

                    for state, count in counts.items():
                        for qubit, bit in enumerate(reversed(state)):
                            density_matrices_individual_qubits[qubit][int(bit), int(bit)] += count / total_shots

                    # Normalize density matrices
                    for qubit in range(num_qubits):
                        density_matrices_individual_qubits[qubit] /= np.trace(density_matrices_individual_qubits[qubit])

                    # Display or use the density matrices as needed
                    # for qubit, density_matrix in density_matrices_individual_qubits.items():
                    #     print(f"Density Matrix for Qubit {qubit}:\n{density_matrix}")


                    rho = density_matrices_individual_qubits[0]

                    for j in range(12):
                        # if i != j:
                        data2 = custom_x[j]
                        normalized_data2 = (np.array(data2) - np.min(data2)) / (np.max(data2) - np.min(data2))
                        angles2 = 2 * np.pi * normalized_data2
                        feature_map_state2 = ZFeatureMap(8)
                        feature_map_circuit2 = feature_map_state2.bind_parameters(dict(zip(feature_map_state2.parameters, angles2)))
                        ansatz_state2 = QuantumCircuit(8, name="Ansatz")
                        ansatz_state2.compose(c1_circuit, list(range(8)), inplace=True)
                        ansatz_state2.compose(p1_circuit, list(range(8)), inplace=True)
                        ansatz_state2.compose(c2_circuit, list(range(4,8)), inplace=True)
                        ansatz_state2.compose(p2_circuit, list(range(4, 8)), inplace=True)
                        ansatz_state2.compose(c3_circuit, list(range(6, 8)), inplace=True)
                        ansatz_state2.compose(p3_circuit, list(range(6, 8)), inplace=True)
                        full_circuit2 = QuantumCircuit(8)
                        full_circuit2.compose(feature_map_circuit2, range(8), inplace=True)
                        full_circuit2.compose(ansatz_state2, range(8), inplace=True)
                        c2 = ClassicalRegister(1, name='c')
                        full_circuit2.add_register(c2)
                        full_circuit2.measure(7, 0)

                        transpiled_circuit2 = transpile(full_circuit2, backend)

                        # Execute the transpiled circuit on the target backend
                        job2 = execute(transpiled_circuit2, backend, shots=1024)

                        # Get the result
                        result2 = job2.result()
                        counts2 = result2.get_counts()

                        num_qubits = 1
                        density_matrices_individual_qubits2 = {qubit: np.zeros((2, 2), dtype=complex) for qubit in range(num_qubits)}

                        # Calculate density matrices for individual qubits
                        total_shots = sum(counts2.values())

                        for state, count in counts2.items():
                            for qubit, bit in enumerate(reversed(state)):
                                density_matrices_individual_qubits2[qubit][int(bit), int(bit)] += count / total_shots

                        # Normalize density matrices
                        for qubit in range(num_qubits):
                            density_matrices_individual_qubits2[qubit] /= np.trace(density_matrices_individual_qubits2[qubit])

                        # Display or use the density matrices as needed
                        # for qubit, density_matrix in density_matrices_individual_qubits.items():
                        #     print(f"Density Matrix for Qubit {qubit}:\n{density_matrix}")


                        sigma= density_matrices_individual_qubits2[0]
                        # print(np.trace(rho), np.trace(sigma))


                        fidelity = calculate_fidelity(rho, sigma)
                        if j >= 6:
                            file.write(f" {x}_{i+1}_{y}_{j-5} {fidelity}\n")
                        elif j < 6:
                            file.write(f" {x}_{i+1}_{x}_{j+1} {fidelity}\n")




                        # if j >= 6:
                        #     print("\nState Fidelity between Student",x,'image',i+1, " and Student",y,'image',j-5, 'is', fidelity)

                        # elif j< 6:
                        #     print("\nState Fidelity between Student",x,'image',i+1, " and Student",x,'image',j+1, 'is', fidelity)
